# 특수 OutputParser 활용법: PandasDataFrame, Datetime, Enum

이 노트북에서는 LangChain의 특수한 용도를 위한 세 가지 OutputParser를 다룹니다:
- **PandasDataFrameOutputParser**: DataFrame에 대한 동적 쿼리 실행
- **DatetimeOutputParser**: 날짜/시간 형식 파싱
- **EnumOutputParser**: 열거형 값으로 파싱

## 학습 목표
- PandasDataFrameOutputParser를 사용하여 자연어로 DataFrame 쿼리하기
- DatetimeOutputParser로 다양한 날짜 형식 처리하기
- EnumOutputParser를 통한 타입 안전한 분류 작업 구현

In [1]:
# 환경 변수 설정
from dotenv import load_dotenv

load_dotenv()

True

## 1. PandasDataFrameOutputParser

PandasDataFrameOutputParser는 자연어 질문을 Pandas DataFrame 연산으로 변환해주는 강력한 도구입니다.

### 주요 특징
- 자연어를 DataFrame 쿼리로 변환
- 열 선택, 행 선택, 집계 연산 지원
- GPT-3.5-turbo 모델 사용 권장

### 작동 방식
파서는 자연어 입력을 받아 Pandas의 쿼리 문법으로 변환하여 실행합니다.

In [2]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
# pandas는 gpt-3.5-turbo 모델 사용을 권장
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [4]:
# 출력 결과를 보기 좋게 포맷팅하는 함수
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    """파서 출력을 보기 좋게 포맷팅합니다."""
    # 파서 출력의 키들을 순회합니다
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

### 데이터 준비

In [5]:
# 타이타닉 데이터셋 로드 (예시)
# 실제 경로는 환경에 맞게 수정 필요
df = pd.DataFrame({
    'PassengerId': [1, 2, 3, 4, 5],
    'Survived': [0, 1, 1, 1, 0],
    'Pclass': [3, 1, 3, 1, 3],
    'Name': ['Braund, Mr. Owen Harris', 
             'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 
             'Heikkinen, Miss. Laina',
             'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
             'Allen, Mr. William Henry'],
    'Sex': ['male', 'female', 'female', 'female', 'male'],
    'Age': [22.0, 38.0, 26.0, 35.0, 35.0],
    'SibSp': [1, 1, 0, 1, 0],
    'Parch': [0, 0, 0, 0, 0],
    'Ticket': ['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450'],
    'Fare': [7.2500, 71.2833, 7.9250, 53.1000, 8.0500],
    'Cabin': [None, 'C85', None, 'C123', None],
    'Embarked': ['S', 'C', 'S', 'S', 'S']
})
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 파서 설정 및 포맷 지시사항

In [6]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

### 열(Column) 조회 예시

In [7]:
# 열 작업 예시
df_query = "Age column 을 조회해 주세요."

# 프롬프트 템플릿을 설정합니다
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n",
    input_variables=["question"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 출력
format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0}}


### 집계 연산 예시

In [8]:
# 직접 계산: row 0 ~ 4의 평균 나이
df["Age"].head().mean()

31.2

In [9]:
# LLM을 통한 계산
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

{'mean': 31.2}


## 2. DatetimeOutputParser

`DatetimeOutputParser`는 LLM의 출력을 `datetime` 형식으로 파싱하는 데 사용할 수 있습니다.

### 주요 날짜 형식 코드

| 형식 코드 | 설명                | 예시          |
|------------|---------------------|---------------|
| %Y         | 4자리 연도          | 2024          |
| %y         | 2자리 연도          | 24            |
| %m         | 2자리 월            | 07            |
| %d         | 2자리 일            | 04            |
| %H         | 24시간제 시간       | 14            |
| %I         | 12시간제 시간       | 02            |
| %p         | AM 또는 PM          | PM            |
| %M         | 2자리 분            | 45            |
| %S         | 2자리 초            | 08            |

In [10]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [11]:
# 날짜 및 시간 출력 파서 설정
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 포맷 지시사항 확인
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%d'.

Examples: 0793-03-01, 1003-06-10, 1892-12-14

Return ONLY this string, no other words!


In [12]:
# 프롬프트 템플릿 생성
template = """Answer the users question:

#Format Instructions: 
{format_instructions}

#Question: 
{question}

#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },
)

In [13]:
# Chain 생성 및 실행
chain = prompt | ChatOpenAI() | output_parser

# 체인을 호출하여 질문에 대한 답변을 받습니다
output = chain.invoke({"question": "Google 이 창업한 연도"})

print(f"파싱된 날짜: {output}")
print(f"문자열 형식: {output.strftime('%Y-%m-%d')}")

파싱된 날짜: 1998-09-04 00:00:00
문자열 형식: 1998-09-04


## 3. EnumOutputParser

EnumOutputParser는 언어 모델의 출력을 미리 정의된 열거형(Enum) 값 중 하나로 파싱하는 도구입니다.

### 주요 특징
- **열거형 파싱**: 문자열 출력을 미리 정의된 Enum 값으로 변환
- **타입 안전성**: 파싱된 결과가 반드시 정의된 Enum 값 중 하나임을 보장
- **유연성**: 공백이나 줄바꿈 문자를 자동으로 처리

In [14]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

In [15]:
# Enum 클래스 정의
class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

In [16]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)

# 포맷 지시사항 확인
print(parser.get_format_instructions())

Select one of the following options: RED, GREEN, BLUE


In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

# 체인 생성
chain = prompt | ChatOpenAI() | parser

In [18]:
# 체인 실행
response = chain.invoke({"object": "하늘"})

print(f"응답: {response}")
print(f"타입: {type(response)}")
print(f"값: {response.value}")

응답: Colors.BLUE
타입: <enum 'Colors'>
값: 파란색


### 다른 예시들

In [19]:
# 여러 물체에 대한 색상 분류
objects = ["토마토", "잔디", "바다"]

for obj in objects:
    response = chain.invoke({"object": obj})
    print(f"{obj}: {response} ({response.value})")

토마토: Colors.RED (빨간색)
잔디: Colors.GREEN (초록색)
바다: Colors.BLUE (파란색)


## 실습 예제: 종합 활용

세 가지 파서를 활용한 실제 사용 예제를 만들어 봅시다.

In [20]:
# 감정 분석을 위한 Enum 정의
class Sentiment(Enum):
    POSITIVE = "긍정적"
    NEGATIVE = "부정적"
    NEUTRAL = "중립적"

# 감정 분석 파서
sentiment_parser = EnumOutputParser(enum=Sentiment)

# 감정 분석 프롬프트
sentiment_prompt = PromptTemplate.from_template(
    """다음 문장의 감정을 분석해주세요:

문장: {text}

Instructions: {instructions}"""
).partial(instructions=sentiment_parser.get_format_instructions())

# 감정 분석 체인
sentiment_chain = sentiment_prompt | ChatOpenAI() | sentiment_parser

In [21]:
# 테스트 문장들
test_sentences = [
    "오늘 날씨가 정말 좋네요!",
    "비가 와서 우울해요.",
    "내일은 수요일입니다."
]

for sentence in test_sentences:
    sentiment = sentiment_chain.invoke({"text": sentence})
    print(f"문장: {sentence}")
    print(f"감정: {sentiment.name} ({sentiment.value})\n")

문장: 오늘 날씨가 정말 좋네요!
감정: POSITIVE (긍정적)

문장: 비가 와서 우울해요.
감정: NEGATIVE (부정적)

문장: 내일은 수요일입니다.
감정: NEUTRAL (중립적)



## 요약 및 활용 가이드

### 1. PandasDataFrameOutputParser
- **사용 시나리오**: 자연어로 데이터 분석을 수행하고 싶을 때
- **장점**: SQL을 모르는 사용자도 데이터 쿼리 가능
- **주의사항**: GPT-3.5-turbo 모델 권장

### 2. DatetimeOutputParser
- **사용 시나리오**: 텍스트에서 날짜/시간 정보를 추출할 때
- **장점**: 다양한 날짜 형식 지원
- **활용**: 일정 관리, 이벤트 파싱, 타임라인 생성

### 3. EnumOutputParser
- **사용 시나리오**: 분류 작업, 감정 분석, 카테고리 지정
- **장점**: 타입 안전성, 제한된 선택지 보장
- **활용**: 텍스트 분류, 상태 판단, 레이블링 작업

### 선택 가이드
- 데이터 분석 → PandasDataFrameOutputParser
- 시간 정보 추출 → DatetimeOutputParser
- 분류/카테고리화 → EnumOutputParser

## 고급 활용: 파서 조합하기

여러 파서를 조합하여 복잡한 출력 구조를 만들 수 있습니다.

In [22]:
# 복합 분석 예제
text = "2020년 3월 11일, WHO가 팬데믹을 선언했습니다. 많은 사람들이 불안해했습니다."

# 1. 날짜 추출
date_prompt = PromptTemplate.from_template(
    "다음 텍스트에서 날짜를 추출하세요: {text}\n\n{format_instructions}"
).partial(format_instructions=output_parser.get_format_instructions())

date_chain = date_prompt | ChatOpenAI() | output_parser
extracted_date = date_chain.invoke({"text": text})

# 2. 감정 분석
sentiment = sentiment_chain.invoke({"text": text})

# 3. 데이터 분석 (예시)
df_analysis = chain.invoke({"question": "What is the average Age?"})

print("분석 결과:")
print(f"- 날짜: {extracted_date.strftime('%Y-%m-%d')}")
print(f"- 감정: {sentiment.name} ({sentiment.value})")
print(f"- 관련 데이터: {extracted_date.year}년 평균 나이는 {df_analysis['mean']}세입니다.")

분석 결과:
- 날짜: 2020-03-11
- 감정: NEGATIVE (부정적)
- 관련 데이터: 2020년 평균 나이는 31.2세입니다.
